### 5회 기출 - 작업2유형
- 벤츠 차량 가격 예측 : https://www.kaggle.com/datasets/mysarahmadbhat/mercedes-used-car-listing 
- train = https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e5_p2_train_.csv 
- test = https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e5_p2_test_.csv
- 예측 변수 price, test.csv에 대해 price 값을 예측하여 제출, 제출 데이터 컬럼은 price만 존재해야함.
- 평가지표는 rmse

In [ ]:
import pandas as pd

train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e5_p2_train_.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e5_p2_test_.csv')

train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)

풀이!

In [ ]:
import sklearn.metrics
print(dir(sklearn.metrics))

['ConfusionMatrixDisplay', 'DetCurveDisplay', 'DistanceMetric', 'PrecisionRecallDisplay', 'PredictionErrorDisplay', 'RocCurveDisplay', 'SCORERS', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_base', '_classification', '_dist_metrics', '_pairwise_distances_reduction', '_pairwise_fast', '_plot', '_ranking', '_regression', '_scorer', 'accuracy_score', 'adjusted_mutual_info_score', 'adjusted_rand_score', 'auc', 'average_precision_score', 'balanced_accuracy_score', 'brier_score_loss', 'calinski_harabasz_score', 'check_scoring', 'class_likelihood_ratios', 'classification_report', 'cluster', 'cohen_kappa_score', 'completeness_score', 'confusion_matrix', 'consensus_score', 'coverage_error', 'd2_absolute_error_score', 'd2_pinball_score', 'd2_tweedie_score', 'davies_bouldin_score', 'dcg_score', 'det_curve', 'euclidean_distances', 'explained_variance_score', 'f1_score', 'fbeta_score', 'fowlkes_mallows_score', 'ge

In [ ]:
# [0] import 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_squared_log_error as msle

# msle, rmsle -> 음수가 있어서는 안됨
# rmse, rmsle -> sklearn.metrics에 없음 
# -> mse ** 0.5 -> rmse, msle ** 0.5 -> rmsle
def get_scores(model, x_train, x_test, y_train, y_test):
    A = model.score(x_train, y_train)
    B = model.score(x_test, y_test)
    y_pred = model.predict(x_test)
    #y_pred[0] = -1
    y_pred = np.where(y_pred > 0, y_pred, -y_pred)  # 음수처리 방법
    C = mse(y_test, y_pred) ** 0.5
    D = mape(y_test, y_pred)
    E = msle(y_test, y_pred) ** 0.5  # 음수가 있어서는 안됨
    return f'{A:.4f} {B:.4f} rmse : {C:.4f}, mape: {D:.4f}, rmsle {E:.4f}'

# ValueError: Mean Squared Logarithmic Error cannot be used when targets contain negative values.


# [1] 파일 불러오기
X_use = pd.read_csv('x_train.csv')
Y_use = pd.read_csv('y_train.csv')
X_submission = pd.read_csv('x_test.csv')

dfX = pd.concat([X_use, X_submission])

dfX2 = pd.get_dummies(dfX)

Y = Y_use['price']

# [4] X_useF, X_submissionF 로 분리
X_useF = dfX2.iloc[:len(X_use)]
X_submissionF = dfX2.iloc[len(X_use):]

# [5] train_test_split 
# stratify 는 분류모델에서 사용!  분류모델에서도 범주별 데이터가 2개 이상이어야 함
temp = train_test_split(X_useF, Y, test_size=0.25, random_state=0)
x_train, x_test, y_train, y_test = temp
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape) 
# (7367, 5) (2456, 5) (7367,) (2456,)

model4 = RandomForestRegressor(random_state=1234)
model4.fit(x_train, y_train)
print(get_scores(model4, x_train, x_test, y_train, y_test))

submission = pd.DataFrame()
submission['price'] = model4.predict(X_submissionF)
submission.to_csv('수험번호.csv', index=False)

# temp = pd.read_csv('수험번호.csv')
# print(temp.shape)
# print(temp.head())  # model             26  transmission       4  fuelType           4

ID              9823
model             26
year              25
transmission       4
mileage         7550
fuelType           4
tax               38
mpg              116
engineSize        26
dtype: int64
ID              3296
model             22
year              21
transmission       3
mileage         2788
fuelType           3
tax               29
mpg               94
engineSize        21
dtype: int64
(7367, 41) (2456, 41) (7367,) (2456,)
0.9936 0.9445 rmse : 2594.9213, mape: 0.0676, rmsle 0.0998


In [ ]:
# print(X_use.nunique())
# print(X_submission.nunique())
X_use.transmission.unique()

array(['Automatic', 'Semi-Auto', 'Manual', 'Other'], dtype=object)

In [ ]:
X_submission.transmission.unique()

array(['Automatic', 'Manual', 'Semi-Auto'], dtype=object)

###----------------------`

In [ ]:
# [0] import 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor  
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_percentage_error as mape

def get_scores(model, x_train, x_test, y_train, y_test):
    A = model.score(x_train, y_train)
    B = model.score(x_test, y_test)
    y_pred = model.predict(x_test)
    C = mse(y_test, y_pred) ** 0.5
    D = mape(y_test, y_pred)
    return '{:.4f} {:.4f} rmse : {:.4f}, mape: {:.4f}'.format(A, B, C, D)

# [1] 파일 불러오기
X_use = pd.read_csv('x_train.csv')
Y_use = pd.read_csv('y_train.csv')
X_submission = pd.read_csv('x_test.csv')
#print(X_use.shape, Y_use.shape, X_submission.shape)
# (9823, 9) (9823, 2) (3296, 9)

# [2] 데이터 탐색
# [2-1] info
#X_use.info()   # object : 'model', 'transmission', 'fuelType'
#Y_use.info()
#X_submission.info()

# [2-2] describe
# feature = ['year', 'mileage', 'tax', 'mpg', 'engineSize']
# print(X_use[feature].describe())
# print(X_submission[feature].describe())

# [2-3] object
# ANOVA 검정을 할 수 있음 
# 'model', 'transmission', 'fuelType'
# X_use에는 존재하는 범주인데, X_submission에는 존재하지 않는다면?
# - X_use에서 제거 (학습 방해요인 제거)
# 'model' -> 종류가 많고, X_use, X_submission 의 종류가 다음 X_submission 220
# => 모델의 범주화
# 'transmission', 'fuelType' -> X_use 에 'Other'가 있는데 X_submission에는 없다
# => 삭제고민
# X_use['fuelType'].value_counts(normalize=False)

# LabelEncoding, OneHotEncoding
# 'model' -> 가격 평균이 다름 -> 가격평균이 낮은 모델 -> 높은 순서에 맞추서 범주화

# [2-4] corr
dfXY = pd.merge(X_use, Y_use, on='ID')
corr = dfXY.corr(numeric_only=True)
#print(corr)

#print(X_use.describe())  - 스케일링

# [3] X, Y 변수 선택
dfX = pd.concat([X_use, X_submission])

dfX['transmission'] = dfX['transmission'].astype('category').cat.codes
dfX['fuelType'] = dfX['fuelType'].astype('category').cat.codes
dfX['model'] = dfX['model'].astype('category').cat.codes

temp = pd.get_dummies(dfX[['transmission', 'fuelType', 'model']])

feature = ['year', 'mileage', 'tax', 'mpg', 'engineSize']
feature2 = ['transmission', 'fuelType', 'model']
X = pd.concat([dfX[feature], temp], axis=1)  # 0.9930 0.9420 rmse : 2836.0253, mape: 0.0672
#X = dfX[feature + feature2]  # 0.9930 0.9420 rmse : 2836.0253, mape: 0.0672
#X = dfX[feature]
Y = Y_use['price']
#print(X.shape, Y.shape)

# scaler = MinMaxScaler()
# X = pd.DataFrame(scaler.fit_transform(X))

# [4] X_useF, X_submissionF 로 분리
X_useF = X.iloc[:len(X_use)]
X_submissionF = X.iloc[len(X_use):]
#print(X_useF.shape, X_submissionF.shape, X_submission.shape)   # (9823, 5) (3296, 5)

# [5] train_test_split 
# stratify 는 분류모델에서 사용!  분류모델에서도 범주별 데이터가 2개 이상이어야 함
temp = train_test_split(X_useF, Y, test_size=0.25, random_state=123 )
x_train, x_test, y_train, y_test = temp
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape) 
# (7367, 5) (2456, 5) (7367,) (2456,)

model1 = LinearRegression()
model1.fit(x_train, y_train)
print(get_scores(model1, x_train, x_test, y_train, y_test))

model2 = KNeighborsRegressor(7)
model2.fit(x_train, y_train)
print(get_scores(model2, x_train, x_test, y_train, y_test))

model3 = DecisionTreeRegressor(max_depth=11, random_state=1234)
model3.fit(x_train, y_train)
print(get_scores(model3, x_train, x_test, y_train, y_test))

model4 = RandomForestRegressor(random_state=1234)
model4.fit(x_train, y_train)
print(get_scores(model4, x_train, x_test, y_train, y_test))

model5 = XGBRegressor(random_state=1234)
model5.fit(x_train, y_train)
print(get_scores(model5, x_train, x_test, y_train, y_test))

model6 = LGBMRegressor(random_state=1234)
model6.fit(x_train, y_train)
print(get_scores(model6, x_train, x_test, y_train, y_test))

submission = pd.DataFrame()
submission['price'] = model4.predict(X_submissionF)

submission.head()

(7367, 8) (2456, 8) (7367,) (2456,)
0.6974 0.7260 rmse : 6161.5182, mape: 0.1806
0.6061 0.4947 rmse : 8367.7882, mape: 0.2258
0.9734 0.9222 rmse : 3283.5502, mape: 0.0857
0.9930 0.9420 rmse : 2836.0253, mape: 0.0672
0.9871 0.9289 rmse : 3139.5029, mape: 0.0648
0.9654 0.9281 rmse : 3157.4657, mape: 0.0704


,price
0,4409.80
1,59469.88
2,16940.82
3,13666.20
4,14960.05


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import numpy as np

X_use = pd.read_csv('x_train.csv')
Y_use = pd.read_csv('y_train.csv')
X_submission = pd.read_csv('x_test.csv')

y = Y_use['price']
x = X_use.drop(columns =['ID'])
test_x = X_submission.drop(columns =['ID'])

dum_x = pd.get_dummies(x)
# train에는 있고, test에는 없는 데이터 존재 컬럼숫자가 다름 -> reindex해줘야함
dum_test = pd.get_dummies(test_x)
dum_test = dum_test.reindex(columns = dum_x.columns, fill_value=0)

x_train,x_test,y_train,y_test = train_test_split(dum_x,y)

rr = RandomForestRegressor()
rr.fit(x_train,y_train)

pred = rr.predict(x_test)
print('test rmse', np.sqrt(mean_squared_error(y_test,pred)))

pred_test = rr.predict(dum_test)

sumission = pd.DataFrame()
sumission['price']  = pred_test
sumission.head()
# sumission.to_csv('00000000.csv',index=Fasle)

In [ ]:
X_use['fuelType'].unique()

array(['Diesel', 'Petrol', 'Hybrid', 'Other'], dtype=object)

In [ ]:
from scipy.stats import f_oneway
dfXY = pd.merge(X_use, Y_use, on='ID')
#print(dfXY.head(3))
print(dfXY.groupby('fuelType')['price'].var())
t = dfXY.groupby('fuelType')['price']
A = t.get_group('Hybrid')
B = t.get_group('Petrol')
C = t.get_group('Diesel')
f_oneway(A, B, C)  # pvalue=3.8144800747197846e-83

In [ ]:
from scipy.stats import f_oneway
dfXY = pd.merge(X_use, Y_use, on='ID')
#print(dfXY.head(3))
print(dfXY.groupby('transmission')['price'].agg(['mean','var']))
t = dfXY.groupby('transmission')['price']
A = t.get_group('Semi-Auto')
B = t.get_group('Automatic')
C = t.get_group('Manual')
f_oneway(A, B, C)  # pvalue=2.0858678338369772e-185

                      mean           var
transmission                            
Automatic     24479.867257  1.638731e+08
Manual        15420.206799  1.761171e+07
Other         15847.500000  1.627351e+07
Semi-Auto     26816.822581  1.270101e+08


F_onewayResult(statistic=444.204989364427, pvalue=2.0858678338369772e-185)

In [ ]:
from scipy.stats import f_oneway
dfXY = pd.merge(X_use, Y_use, on='ID')
#print(dfXY.head(3))
print(dfXY.groupby('model')['price'].agg(['mean','var']).sort_values('mean'))
# t = dfXY.groupby('transmission')['price']
# A = t.get_group('Semi-Auto')
# B = t.get_group('Automatic')
# C = t.get_group('Manual')
# f_oneway(A, B, C)  # pvalue=2.0858678338369772e-185